<a href="https://colab.research.google.com/github/ram130849/Deep_Learning_Systems_Assignments/blob/main/TensorFlow/Ramki/E533_DLS_Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install IPython
!pip install librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 4.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import numpy as np
from IPython.display import Audio
import librosa
from google.colab import drive
import tensorflow as tf
from tensorflow.keras import backend as K
from librosa.core import stft,istft
from tensorflow.keras.layers import Conv1D,Conv2D,MaxPooling1D,MaxPooling2D,Dropout,Dense,Flatten
import matplotlib.pyplot as plt
import os
import soundfile as sf
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [6]:
# !unzip '/content/drive/MyDrive/DLS_DATA/homework3.zip' -d '/content/drive/MyDrive/DLS_DATA/timit-hw3'

In [16]:
path = '/content/drive/MyDrive/DLS_DATA/timit-hw3/timit-homework/tr'

X = []
S = []
N = []

for f in os.listdir(path):
    if(os.path.isfile(os.path.join(path,f))):
        if(f[:3]=='trn'):
          s, sr = librosa.load(os.path.join(path,f),sr=None)
          trn_stft=librosa.stft(s, n_fft=1024, hop_length=512)
          N.append(abs(trn_stft))
        elif(f[:3]=='trs'):
          s, sr = librosa.load(os.path.join(path,f),sr=None)
          trs_stft=librosa.stft(s, n_fft=1024, hop_length=512)
          S.append(abs(trs_stft))
        else:
          s, sr = librosa.load(os.path.join(path,f),sr=None)
          trx_stft=librosa.stft(s, n_fft=1024, hop_length=512)
          X.append(abs(trx_stft))
print(len(N))
print(len(S))
print(len(X))

1200
1200
1200


In [22]:
print(N[-1].shape)
print(S[-1].shape)
print(X[-1].shape)

(513, 111)
(513, 92)
(513, 119)


In [47]:
M_arr = []
for i in range(len(N)):
    # print("Before:",S[i].shape,N[i].shape)
    if(S[i].shape[1]<N[i].shape[1]):
        S[i] = np.pad(S[i],((0,0),(0,N[i].shape[1]-S[i].shape[1])),mode='constant')
    elif(S[i].shape[1]>N[i].shape[1]):
        N[i] = np.pad(N[i],((0,0),(0,S[i].shape[1]-N[i].shape[1])),mode='constant')
    # print("After:",S[i].shape,N[i].shape)
    M = np.where((S[i]>N[i]),1,0)
    M_arr.append(M)

In [48]:
print(len(M_arr))

1200
